In [1]:
import requests
from bs4 import BeautifulSoup
import gradio as gr
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM
from langchain.chains import RetrievalQA
from langchain.docstore.document import Document

def extract_text_from_url(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    page_text = soup.get_text()
    print(page_text[:1000])
    return page_text

def split_text(text, chunk_size=500):
    return [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]


def create_qa_chain():
    # LLM
    llm = OllamaLLM(model="llama3.2", server_url="http://localhost:11434")
    embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vectorstore = Chroma(persist_directory="./vectorstore", embedding_function=embedding_model)

    # URL 
    url = "https://es.wikinews.org/wiki/Crisis_pol%C3%ADtica_hist%C3%B3rica_en_Francia_por_ca%C3%ADda_de_Michel_Barnier"


    # Extraer y procesar el contenido
    page_content = extract_text_from_url(url)
    chunks = split_text(page_content)
    print(f"Number of chunks: {len(chunks)}")

    documents = [Document(page_content=chunk) for chunk in chunks]
    vectorstore.add_documents(documents)
    print("Documents added to the vector store.")

    # Crear el prompt de la cadena de pregunta-respuesta
    prompt = ChatPromptTemplate.from_template(
        template="Use the context below to answer the user's question:\n\n{context}\n\nQuestion: {question}\nAnswer:"
    )

    # Crear un retriever
    retriever = vectorstore.as_retriever()

    # Crear la cadena de QA
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever,
        chain_type_kwargs={"prompt": prompt}
    )

    return qa_chain

# Crear la cadena de QA al inicio
qa_chain = create_qa_chain()

def answer_question(question):
    print(f"Processing question: {question}")  #  pregunta procesada
    result = qa_chain.invoke(question)
    print(f"Answer: {result}")  # respuesta generada
    return result


# Interfaz Gradio
iface = gr.Interface(fn=answer_question,
                     inputs="text",
                     outputs="text",
                     live=True,
                     title="Wikipedia",
                     description="Crisis política histórica en Francia por caída de Michel Barnier, qué quieres saber?.")

# Ejecutar la interfaz 
iface.launch()



/home/bigdata/miniconda3/envs/rag/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm






Crisis política histórica en Francia por caída de Michel Barnier - Wikinoticias




































Ir al contenido







Menú principal





Menú principal
mover a la barra lateral
ocultar



		Navegación
	


Página principalSala de redacciónCambios recientesPágina aleatoriaAyuda





		regiones
	


ÁfricaAsiaAntillasCentroaméricaEuropaMedio OrienteNorteaméricaOceaníaSudamérica





		wikinoticias
	


CaféPolíticas


















Buscar











Buscar






















Apariencia
















Donaciones

Crear una cuenta

Acceder








Herramientas personales





Donaciones Crear una cuenta Acceder





		Páginas para editores desconectados más información



ContribucionesDiscusión




























Contenidos
mover a la barra lateral
ocultar




Inicio





1
Fuentes


















Cambiar a la tabla de contenidos







Crisis política histórica en Francia por caída de Michel Barnier



Añadir idiomas





Añadir enlaces













Processing question: q
Answer: {'query': 'q', 'result': 'La crisis política en Francia se debió a la caída de Michel Barnier, primer ministro, debido a una polémica suscitada por un proyecto de ley presupuestaria.'}
Processing question: quién es Michael Barnier?
Answer: {'query': 'quién es Michael Barnier?', 'result': 'Michel Barnier es un político francés que ha sido el primer ministro de Francia desde 2017.'}
